# DABN13 - Assignment 2
## Preamble: Predicting purchases in online shops.
This assignment will be based on a dataset on online shopper purchase data which is available on the UC Irvine Machine Learning Library. A description of all variables is available [here  ](https://www.kaggle.com/henrysue/online-shoppers-intention). Among the 11 variables in the dataset, we will only use the three following:

 - **Revenue**: (TRUE/FALSE) Whether a purchase was made by a visitor to the online shop.
 - **ProductRelated_Duration**: (numerical) Time spend on pages relevant/related to the product in question.
 - **ExitRates**: (numerical) The percentage of visits to the online shop that end with visiting the site of the product at issue.


In [5]:
import numpy as np
import pandas as pd
import os
import pyreadr as prdr

## Part 1: Logistic regression with `sklearn`

In this basic part, we are getting some experience with using scikit-learn to learn logistic regressions. In the steps below, we will train a quite small logistic regression model with `Revenue` as output variable and the following inputs (in addition to the intercept):

1. ` ExitRate ` without further transformation
2. The (natural) logarithm of ` ProductRelated_Duration + 1 `

### Task 1a)
First, we need to prepare our data. Conduct the following steps:

1. Load the data into python and save it in an object called `shoppers`. The dataset is contained in a comma-separated spreadsheet. Accordingly, you will need to use the `read_csv()` command in Pandas.
2. Use the `columns` method on `shoppers` to change the variable name of `ExitRates` to `ER`. More specifically, you will need to rename a particular column of `shoppers`.
3. Create a new variable `lPR_Dur` inside `shoppers` that contains the (natural) logarithm of  `ProductRelated_Duration + 1`.

In [12]:
# CODE_CHUNK code_chunk_01
# os.chdir('C:\\Users\\??') # Change working directory if needed

# 1.
shoppers = pd.read_csv('online_shoppers_intention.csv')

# 2.
shoppers = shoppers.rename(columns={'ExitRates': 'ER'})

# 3.
shoppers['lPR_Dur'] = np.log(shoppers['ProductRelated_Duration']+1)

### Task 1b)

Technically, we could use the `statsmodels` package to learn a logistic regressions. However, the functionality of `statsmodels` addresses applications in classical statistics rather than machine learning. Since we want to use logistic regression as a supervised learning algorithm in order to predict the output of new data points, it is much wiser to switch to scikit-learn (`sklearn`). `sklearn` provides you with simple functions that implement model training, tuning and validation and therefore covers the entire spectrum of standard methods in supervised learning.

Learning a logistic regression with `sklearn` works almost identically as with `statsmodels`. Do it as follows:

1. Create a  Pandas series `y_1b` that contains the `Revenue` variable from `shoppers`
2. Create a similar data frame `X_1b` whose columns are  ` ER `, ` lPR_Dur ` and the square of `lPR_Dur`.
3. Instantiate a logistic regression model using the `LogisticRegression()` function from the `linear_model` module and save this model specification as `glm_spec_1b`. The function has an argument `penalty`. Set this argument to `None`. Furthermore ensure that an intercept is automatically added to the input variables.
4. Apply the `fit()`-method to `glm_spec_1b` to learn the model and save the learned model as `glm_fit_1b`. Here, specify that the model is learned using with inputs `X_1b` and output `y_1b`.


In [17]:
# CODE_CHUNK code_chunk_02
# 1.
y_1b = pd.Series(shoppers["Revenue"])

# 2.
X_1b = pd.DataFrame({
    'ER': shoppers['ER'],
    'lPR_Dur': shoppers['lPR_Dur'],
    'lPr_Dur2': shoppers['lPR_Dur']**2
})

# 3.
from sklearn.linear_model import LogisticRegression
glm_spec_1b = LogisticRegression(penalty = None, fit_intercept=True)

# 4.
glm_fit_1b  = glm_spec_1b.fit(X_1b, y_1b)

print(glm_fit_1b.coef_)

[[-32.63987528  -0.58940172   0.06451171]]


### Task 1c)
A fundamental principle of machine learning is that we divide the data available to us into different sets which we use for learning, model tuning and algorithm choice.

`sklearn` makes this very simple for us since its `model_selection`-module contains the `train_test_split()` function. It returns splitted versions of all objects that you provide as inputs. I prepared a preliminary code chunk below that shows how you need to specify the desired splitted objects. Complete the code chunk by entering the inputs to `train_test_split()`. More specifically, provide

1. the data objects before splitting in correct order,
2. an argument that allocates 50% of all data points to the training data,
3. an argument that sets the initial state of the random number generator to 3 (we need this to ensure replicability).


In [18]:
# CODE_CHUNK code_chunk_03
from sklearn.model_selection import train_test_split

X_1b_train, X_1b_test, y_1b_train, y_1b_test = train_test_split(X_1b, y_1b,train_size=0.5, random_state=3)

### Task 1d)
Now, please refit the model from Task 1b using only your training data from Task 1c. Save the resulting learned model as `glm_fit_1d`.

In [20]:
# CODE_CHUNK code_chunk_04
glm_fit_1d =  glm_spec_1b.fit(X_1b_train, y_1b_train)

print(glm_fit_1d.coef_)

[[-32.23959758  -0.68440669   0.07091558]]


### Task 1e)
Now that we have fitted our model, we want to evaluate its predictive performance on hold-out validation data. We have already created this test data in Task 1c as `y_1b_test` and `X_1b_test.`

Before we can evaluate model performance, we first need to obtain predicted probabilities for purchases on the test data. Use the ` predict_proba() `-method on `glm_fit_1d` to obtain such predicted conditional probabilities from the model fit in Task 1d on the observations in our test set. Save them as `glm_prob_1e`

According to the sklearn documentation, the columns of `glm_prob_1e` contain class probabilities where classes are ordered as they are in `glm_fit_1d.classes_`. Which column index contains the probabilities for class `True`? Write your answer into the string variable `which_column_truepreds_1e`



In [ ]:
# CODE_CHUNK code_chunk_05
glm_prob_1e = glm_fit_1d.predict_proba(X_1b_test)
which_column_truepreds_1e = "The probabilities are contained in the second column, so column 1 (since python is 0-indexed)-"

### Task 1f)
In a next step, we apply a classification rule to map our predicted probabilities into class predictions. Our classification rule is to predict the most likely class.

First, create a new vector ` glm_pred_1f ` which has as many elements as ` glm_prob_1e ` and which consists entirely of the logical statement `False` (without citation marks!)

Second, replace `False` in ` glm_pred_1f ` with `True` for all elements where the corresponding predicted probability for category `True` exceeds the threshold used for the classifier mentioned above. You can do this by indexing `glm_pred_1f` using square brackets. Simply write a true-or-false (or logical) statement in the square brackets. For rows where it is true, the value of ` glm_pred_1f ` will be changed.

Please additionally write the true-or-false statement that you use into the string variable ` logical_1f ` for the sake of making assignment evaluation simpler for us.

*Note:* We could even get class predictions by applying the `predict()`-method to `glm_fit_1d`. However, that does not allow us to fully control the classification rule.

In [24]:
# CODE_CHUNK code_chunk_06
# 1.
glm_pred_1f = np.zeros(glm_prob_1e.shape[0], dtype=bool)

# 2.
glm_pred_1f[glm_prob_1e[:,1]>0.5] = True

print(np.unique(glm_pred_1f, return_counts=True)) # Run this without modification

# 3.
logical_1f = "glm_prob_1e[:,1]>0.5"

(array([False,  True]), array([6150,   15]))


### Task 1g)
Choose an appropriate error function and write its name in the string variable `chosenerrfun_1g`. Then, use the objects created in the previous tasks of this part to obtain (overall) test error for the logistic regression model fitted in Task 1d.

In [25]:
# CODE_CHUNK code_chunk_07
chosenerrfun_1g = "Missclassification rate"
testerr_1g      = np.mean(glm_pred_1f != y_1b_test)
print(testerr_1g)

0.15506893755068937


## Part 2: Class-specific prediction errors

This part is more advanced than parts 1 and 3. In classification problems, overall test error may not always be our primary concern. To get a more differentiated picture, confusion matrices and the ROC curve are useful tools. We will get both using the metric module of Scikit-learn which provides a large number of performance criteria for binary classification.


### Task 2a)
A basic confusion matrix can easily be obtained using the `confusion_matrix` function of `sklearn.metrics`. This function only requires two inputs:

1. The test outcomes.
2. The predicted class on the test set.

Obtain the confusion matrix and save it as `confumat_2a`.

In [ ]:
# CODE_CHUNK code_chunk_08
from sklearn.metrics import confusion_matrix

confumat_2a = ??
print(confumat_2a)

### Task 2b)
The confusion matrix obtained in the last task is rather rudimentary. For this reason, we are now going to write a function which produces a more luxurious confusion matrix with additional performance measures.

Below, I prepared a function that takes a true outcomes, predicted probabilities for the category of interest and a desired threshold probability for the classification rule as inputs and returns a dictionary object containing the corresponding confusion matrix, TPR, FPR and overall classification error. All that is left for you is the following tasks:

1. Specify the row and column names of the confusion matrix correctly
2. Use the four elements of `cmat` to calculate `FPR`, `TPR` and the classification error `error`.

In [ ]:
# CODE_CHUNK code_chunk_09
def  performancemetrics(y_true, y_prob, cutoff):
    # Don't change the 2 rows below
    y_pred       = np.greater(y_prob,cutoff)
    cmat         = confusion_matrix(y_true,y_pred)

    # 1.
    column_names = ["??", "??"]
    row_names    = ["??", "??"]

    # 2.
    FPR   = ??
    TPR   = ??
    error = ??

    # Don't change the two lines below
    cmat         = pd.DataFrame(cmat, columns=column_names, index=row_names)
    allresults = {'confusion_matrix': cmat, 'FPR':FPR, "TPR":TPR, 'classification_error':error}
    return(allresults)

### Task 2c)

Now save as `metrics_2c` the output of `performancemetrics()` with the prediction object of task 2a and a threshold probability of 50% as input. Additionally, answer two questions:

1. Are you satisfied with the overall accuracy with which our model predicts purchases?
2. Is the accuracy with which observed purchases are correctly predicted satisfactory? Assume here that we have considerable interest in predicting actual purchases correctly.

In [ ]:
# CODE_CHUNK code_chunk_10

metrics_2c = ??
print(metrics_2c)
overall_acc_verdict2c      = "??"
obs_purchase_acc_verdict2c = "??"


### Task 2d)

Assume we would like to get a classifier that has relatively balanced class-specific performance. In other words, we want to choose a threshold such that TPR is approximately 1-FPR. In order to see the trade-offs that are available to us, we will look at a ROC curve.

ROC curves can be plotted using the `RocCurveDisplay.from_predictions()` function in the sklearn metrics module. The inputs to this function are

1. The test outcomes.
2. The predicted probabilities for class 1 on the test set.

Use the code chunk below to plot a ROC curve. Then proceed with the rest of this task in the following text block.

In [ ]:
# CODE_CHUNK code_chunk_11
from sklearn.metrics import RocCurveDisplay

??

Unfortunately, the plotted ROC curve does not indicate the threshold value leading to a particular point on the curve. Fortunately, the information contained in a ROC curve can be obtained using the `roc_curve()` function in the sklearn metrics module.

The inputs to  `roc_curve()` are true outputs and the predicted probabilities for class 1. Use the corresponding series from Part 1 of this assignment as function inputs and save the result as `roc_fpr_2d`, `roc_tpr_2d` and `roc_thresholds_2d`.

Next, find the the threshold that balances class-specific performance best. You could do that by finding the index at which TPR and (1-FPR) are as close are possible. Then you simply return the value of `roc_thresholds_2d` at this index. State your chosen threshold in the  variable `opt_threshold_2d`. Round to two decimals, if needed.

In [ ]:
# CODE_CHUNK code_chunk_12
from sklearn.metrics import ??

# 1.
roc_fpr_2d, roc_tpr_2d, roc_thresholds_2d = roc_curve(??)

# 2.
opt_threshold_2d = ??

print(opt_threshold_2d)


### Task 2e)
To what extend does our chosen threshold from Task 2d compromise overall accuracy? To find this out, use your `performancemetrics()` function from Task 2b and save the output that you get with the threshold chosen in Task 2d as `metrics_2e`.

Then, use the string variable `is_accuracy_compromised_2e` to discuss whether overall accuracy is noteably affected if we switch from a threshold probability of 50% to a value that balances class-specific prediction performance.

In [ ]:
# CODE_CHUNK code_chunk_13
metrics_2e = ??
print(metrics_2e)

is_accuracy_compromised_2e = "??"

## Part 3: Multiclass logistic regression with `sklearn`

We can use the `LogisticRegression` function from `sklearn` to learn models whose output variable has more than two categories. The data we are using to learn a multiclass logistic regression is drug consumption data. Our response variable is usage of drugs (Cocaine , Crack, Ecstasy, and Heroin) and we have three possible responses, "never used", "used more than a year ago", and "used within a year". As explanatory variables we have personality test data, demographic data, and consumption of chocolate, alcohol, and nicotine.

### Task 3a)

Conduct the following steps to prepare your data:

1. Load `drug_train.RDS` using the `read_r()` function in the `pyreadr` package and save it as a data frame `drug_data`.
2. Extract the variable *drugs.usage* into a Pandas series called `y_3a`
3. Create a Pandas dataframe `X_3a` containing all other variables in `drug_data`.  Note that you will need to transform categorical variables into indicator variables, e.g. using the `get_dummies()` function in Pandas.

In [ ]:
# CODE_CHUNK code_chunk_14
# 1.
drug_data = prdr.read_r(??)
drug_data = drug_data[None]

# 2.
y_3a = ??

# 3.
X_3a = ??

### Task 3b)

Specify and learn fit a multinomial logistic regression using `LogisticRegression()`. In the specification, choose no penalty and make sure an intercept is added. Additionally, set the `multi_class` argument to `'multinomial'`. Lastly, increase the number of iterations if this is needed for the learning algorithm to converge.

Save the resulting object as `mlogit_fit_3b`.

In [ ]:
# CODE_CHUNK code_chunk_15
mlogit_fit_3b = ??

### Task 3c)

We now skip the entire data splitting step that we discussed in Part 1. Instead, we say that model `mlogit_fit_3b` is our result of the entire modeling process and it is implemented in practice. We now get new test data for which we obtain output predictions. To prepare this, conduct the following steps:

1. Load *drug_test.RDS* as object `drug_data_test`.
2. Create an input variable matrix `X_3c_test` in the same way as in Task 3a, but using `drug_data_test` instead of `drug_data`.

In [ ]:
# CODE_CHUNK code_chunk_16
drug_data_test = ??
drug_data_test = drug_data_test[None]

X_3c_test = ??

### Task 3d)

Get predictions from model `mlogit_fit_3b` for your test data using the `predict_proba()` and `predict()` methods. Save your predictions as `mlogit_prob_3c` and `mlogit_pred_3c`, respectively.


In [ ]:
# CODE_CHUNK code_chunk_17
mlogit_prob_3c = ??
mlogit_pred_3c = ??

As time passes, you also observe outputs for your first batch of test data. This allows you to evaluate the accuracy of your prediction model during deployment. The variable *drugs.usage* in `drug_data_test` contains these test outputs. Extract them into a vector `ytest_3d`.

Then, use the `confusion_matrix` function in sklearn to get a confusion matrix. Save it as `confumat_3d`.


In [ ]:
# CODE_CHUNK code_chunk_18
ytest_3d    = ??
confumat_3d = ??